In [5]:
import numpy as np
import pandas as pd

Read station metadata from a comma-separated file. Columns are station ID, station name, longitude, latitude, and available water capacity.

In [15]:
metadata = pd.read_csv("data/1012475.metadata")
#metadata.head()
station_id = metadata["Station"][0]
lat = metadata["Lat"][0]
awc = metadata["AWCmm"][0]
print(f"ID: {station_id}, latitide: {lat}, AWC: {awc}")

ID: 1012475, latitide: 48.42, AWC: 123


Read climatology data from a space-delimited file. Columns are station ID, year, month, day, tmMax, tmin, and precipitation.

In [7]:
station_data = pd.read_csv("data/1012475.dat", header=None, delim_whitespace=True)
station_data.head()

,0,1,2,3,4,5,6
0,1012475,1950,1,1,-0.4,-5.4,4.6
1,1012475,1950,1,2,-5.4,-8.9,3.1
2,1012475,1950,1,3,-3.6,-9.5,5.9
3,1012475,1950,1,4,-1.1,-7.0,1.6
4,1012475,1950,1,5,2.8,-5.5,6.9


In [11]:
column_names = {0: "station_id", 1: "year", 2: "month", 3: "day", 4: "tmax", 5: "tmin", 6: "prcp"}
station_data.rename(column_names, axis=1, inplace=True)
station_data.head()

,station_id,year,month,day,tmax,tmin,prcp
0,1012475,1950,1,1,-0.4,-5.4,4.6
1,1012475,1950,1,2,-5.4,-8.9,3.1
2,1012475,1950,1,3,-3.6,-9.5,5.9
3,1012475,1950,1,4,-1.1,-7.0,1.6
4,1012475,1950,1,5,2.8,-5.5,6.9


In [32]:
station_data['date'] = station_data[['year', 'month', 'day']].apply(lambda s : pd.datetime(*s),axis = 1)
station_data.drop(columns=["year", "month", "day"], inplace=True)
station_data.head()

,station_id,year,month,day,tmax,tmin,prcp,datetime,date
0,1012475,1950,1,1,-0.4,-5.4,4.6,1950-01-01,1950-01-01
1,1012475,1950,1,2,-5.4,-8.9,3.1,1950-01-02,1950-01-02
2,1012475,1950,1,3,-3.6,-9.5,5.9,1950-01-03,1950-01-03
3,1012475,1950,1,4,-1.1,-7.0,1.6,1950-01-04,1950-01-04
4,1012475,1950,1,5,2.8,-5.5,6.9,1950-01-05,1950-01-05


,station_id,tmax,tmin,prcp,date
0,1012475,-0.4,-5.4,4.6,1950-01-01
1,1012475,-5.4,-8.9,3.1,1950-01-02
2,1012475,-3.6,-9.5,5.9,1950-01-03
3,1012475,-1.1,-7.0,1.6,1950-01-04
4,1012475,2.8,-5.5,6.9,1950-01-05
5,1012475,6.3,1.2,5.1,1950-01-06
6,1012475,3.9,-0.4,0.0,1950-01-07
7,1012475,3.0,-1.5,5.2,1950-01-08
8,1012475,3.1,-1.9,8.5,1950-01-09
9,1012475,4.9,0.8,6.1,1950-01-10
